# Creating ApoE Bundles from HGVS

HGVS is the de facto standard for manipulating human-readable variation data. This notebook demonstrates the VMC BundleManager, which facilitates translating HGVS expressions to Alleles, Haplotypes, and Genotypes.


```
                             rs7412 
                             NC_000019.10:g.44908822
                             NM_000041.3:c.526
                             C          T
rs429358                 C   APOE-ε4    APOE-ε1
NC_000019.10:g.44908684  T   APOE-ε3    APOE-ε2
NM_000041.3:c.388
```

In [1]:
apoe_alleles = {
    'rs7412C': {'g37': 'NC_000019.9:g.45412079=',
                'g38': 'NC_000019.10:g.44908822=',
                'c': 'NM_000041.3:c.526=',
                'p': 'NP_000032.1:p.Arg176='},
    'rs7412T': {'g37': 'NC_000019.9:g.45412079C>T',
                'g38': 'NC_000019.10:g.44908822C>T',
                'c': 'NM_000041.3:c.526C>T',
                'p': 'NP_000032.1:p.Arg176Cys'},
    'rs429358T': {'g37': 'NC_000019.9:g.45411941=',
                  'g38': 'NC_000019.10:g.44908684=',
                  'c': 'NM_000041.3:c.388=',
                  'p': 'NP_000032.1:p.Cys130='},
    'rs429358C': {'g37': 'NC_000019.9:g.45411941T>C',
                  'g38': 'NC_000019.10:g.44908684T>C',
                  'c': 'NM_000041.3:c.388T>C',
                  'p': 'NP_000032.1:p.Cys130Arg'}
}
apoe_haplotypes = {
    "ε1": ["rs429358C", "rs7412T"],
    "ε2": ["rs429358T", "rs7412T"],
    "ε3": ["rs429358T", "rs7412C"],
    "ε4": ["rs429358C", "rs7412C"],
}

In [2]:
id_function = "serial"   # or: "computed" (default) or "uuid"
from vmc.extra.bundlemanager import BundleManager
bm = BundleManager(id_function=id_function)

# Alleles

In [3]:
for vname, alleles in apoe_alleles.items():
    for h in alleles.values():
        a = bm.add_hgvs_allele(h)

# Haplotypes

In [4]:
haplotypes_hgvs = {
    t: {name: [apoe_alleles[vname][t] for vname in vnames] for name, vnames in apoe_haplotypes.items()}
    for t in "g37 g38 c p".split()
}
haplotypes_hgvs

{'g37': {'ε1': ['NC_000019.9:g.45411941T>C', 'NC_000019.9:g.45412079C>T'],
  'ε2': ['NC_000019.9:g.45411941=', 'NC_000019.9:g.45412079C>T'],
  'ε3': ['NC_000019.9:g.45411941=', 'NC_000019.9:g.45412079='],
  'ε4': ['NC_000019.9:g.45411941T>C', 'NC_000019.9:g.45412079=']},
 'g38': {'ε1': ['NC_000019.10:g.44908684T>C', 'NC_000019.10:g.44908822C>T'],
  'ε2': ['NC_000019.10:g.44908684=', 'NC_000019.10:g.44908822C>T'],
  'ε3': ['NC_000019.10:g.44908684=', 'NC_000019.10:g.44908822='],
  'ε4': ['NC_000019.10:g.44908684T>C', 'NC_000019.10:g.44908822=']},
 'c': {'ε1': ['NM_000041.3:c.388T>C', 'NM_000041.3:c.526C>T'],
  'ε2': ['NM_000041.3:c.388=', 'NM_000041.3:c.526C>T'],
  'ε3': ['NM_000041.3:c.388=', 'NM_000041.3:c.526='],
  'ε4': ['NM_000041.3:c.388T>C', 'NM_000041.3:c.526=']},
 'p': {'ε1': ['NP_000032.1:p.Cys130Arg', 'NP_000032.1:p.Arg176Cys'],
  'ε2': ['NP_000032.1:p.Cys130=', 'NP_000032.1:p.Arg176Cys'],
  'ε3': ['NP_000032.1:p.Cys130=', 'NP_000032.1:p.Arg176='],
  'ε4': ['NP_000032.1:p.Cys

In [5]:
# create protein haplotypes
for t in "g37 g38 c p".split():
    for name, hs in haplotypes_hgvs[t].items():
        h = bm.add_hgvs_haplotype(hs)

In [6]:
for t in "g37 g38 c p".split():
    hht = haplotypes_hgvs[t]
    for h1 in "ε3 ε4".split():
        for h2 in "ε3 ε4".split():
            gt_alleles = (hht[h1], hht[h2])
            g = bm.add_hgvs_genotype(gt_alleles)

In [7]:
bm.as_bundle().as_dict()

{'alleles': {'2': {'id': '2', 'location_id': '1', 'state': 'C'},
  '4': {'id': '4', 'location_id': '3', 'state': 'C'},
  '6': {'id': '6', 'location_id': '5', 'state': 'G'},
  '8': {'id': '8', 'location_id': '7', 'state': 'R'},
  '10': {'id': '10', 'location_id': '9', 'state': 'T'},
  '12': {'id': '12', 'location_id': '11', 'state': 'T'},
  '14': {'id': '14', 'location_id': '13', 'state': 'T'},
  '16': {'id': '16', 'location_id': '15', 'state': 'C'},
  '18': {'id': '18', 'location_id': '17', 'state': 'T'},
  '20': {'id': '20', 'location_id': '19', 'state': 'T'},
  '22': {'id': '22', 'location_id': '21', 'state': 'C'},
  '24': {'id': '24', 'location_id': '23', 'state': 'C'},
  '26': {'id': '26', 'location_id': '25', 'state': 'C'},
  '28': {'id': '28', 'location_id': '27', 'state': 'C'},
  '30': {'id': '30', 'location_id': '29', 'state': 'C'},
  '32': {'id': '32', 'location_id': '31', 'state': 'R'},
  '34': {'id': '34', 'location_id': '33', 'state': 'C'},
  '36': {'id': '36', 'location_id

In [8]:
bm.as_hgvs()

{'alleles': ['NC_000019.9:g.45412079=',
  'NC_000019.10:g.44908822=',
  'NM_000041.3:g.526=',
  'NP_000032.1:g.176=',
  'NC_000019.9:g.45412079C>T',
  'NC_000019.10:g.44908822C>T',
  'NM_000041.3:g.526G>T',
  'NP_000032.1:g.176R>C',
  'NC_000019.9:g.45411941=',
  'NC_000019.10:g.44908684=',
  'NM_000041.3:g.388=',
  'NP_000032.1:g.130=',
  'NC_000019.9:g.45411941T>C',
  'NC_000019.10:g.44908684T>C',
  'NM_000041.3:g.388=',
  'NP_000032.1:g.130C>R',
  'NC_000019.9:g.45411941T>C',
  'NC_000019.9:g.45412079C>T',
  'NC_000019.9:g.45411941=',
  'NC_000019.9:g.45412079C>T',
  'NC_000019.9:g.45411941=',
  'NC_000019.9:g.45412079=',
  'NC_000019.9:g.45411941T>C',
  'NC_000019.9:g.45412079=',
  'NC_000019.10:g.44908684T>C',
  'NC_000019.10:g.44908822C>T',
  'NC_000019.10:g.44908684=',
  'NC_000019.10:g.44908822C>T',
  'NC_000019.10:g.44908684=',
  'NC_000019.10:g.44908822=',
  'NC_000019.10:g.44908684T>C',
  'NC_000019.10:g.44908822=',
  'NM_000041.3:g.388=',
  'NM_000041.3:g.526G>T',
  'NM_000